In [2]:
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Conv2D, Flatten, Dense
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras import backend as K
import keras

import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os import system
import os
import random

import imageio

Using TensorFlow backend.


In [3]:
img_width, img_height = 16, 16

nb_epoch = 50
batch_size = 32

In [4]:
base_dir = '/home/niaki/Code/ImageNet/tiny-imagenet-200'

train_data_dir      = base_dir + '/tiny_train16'
validation_data_dir = base_dir + '/tiny_validation16'
test_data_dir       = base_dir + '/tiny_test16'

train_descrs_dir      = base_dir + '/tiny_sifts/tiny_train16'
validation_descrs_dir = base_dir + '/tiny_sifts/tiny_validation16'
test_descrs_dir       = base_dir + '/tiny_sifts/tiny_test16'

In [5]:
def loading_data(data_dir):
    files = listdir(data_dir + '/class0')
    files.sort()

    images = []

    for file in files:
        image = imageio.imread(data_dir + '/class0/' + file)
    #     image = np.expand_dims(image, axis=0)
        images.append(image)

    images = np.array(images)
    images = images.astype(np.float64) / 255
    print(images.shape)
    return images

In [6]:
x_train = loading_data(train_data_dir)
x_validation = loading_data(validation_data_dir)

(157086, 16, 16, 3)
(3932, 16, 16, 3)


In [7]:
def loading_descrs(descrs_dir):
    files = listdir(descrs_dir + '/class0')
    files.sort()

    descrs = []

    for file in files:
        descr = np.load(descrs_dir + '/class0/' + file)
        descrs.append(descr)

    descrs = np.array(descrs)
    descrs = descrs.astype(np.float64) / 255
    print(descrs.shape)
    return descrs

In [8]:
y_train = loading_descrs(train_descrs_dir)
y_validation = loading_descrs(validation_descrs_dir)

(157086, 128)
(3932, 128)


In [12]:
input_shape = (img_width, img_height, 3)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="relu", padding="same")(input_img)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(128, activation="sigmoid")(x)

encoder = Model(input_img, encoded)

encoder.compile(optimizer='adadelta', metrics=['accuracy'], loss='mean_squared_logarithmic_error')
#next up: encoder.compile(optimizer='sgd', metrics=['categorical_accuracy'], loss='categorical_crossentropy')


encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 16, 16, 3)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 32)          9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4, 4, 32)          9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 2, 2, 32)          0         
__________

In [10]:
def fixed_generator(x_train, y_train, batch_size):
    while True:
        batch_list_x = []
        batch_list_y = []
        
        for i in range(x_train.shape[0]):
            batch_list_x.append(x_train[i])
            batch_list_y.append(y_train[i])
            if len(batch_list_x) == batch_size:
                yield (np.array(batch_list_x),np.array(batch_list_y))
                batch_list_x = []
                batch_list_y = []


In [11]:
model_version = '1.0.0.3_encoder_lastsigmoid_msle_adadelta_metricsa'

os.system('mkdir ' + base_dir + '/weights' + model_version)
checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder.fit_generator(fixed_generator(x_train, y_train, 32),
                steps_per_epoch=157086,
                epochs=50,
                validation_data=fixed_generator(x_validation, y_validation, 32),
                validation_steps=3932,
                callbacks=[checkpointer]
                )

encoder.save(base_dir + '/ae' + model_version + '.h5')

W1025 17:31:24.044293 139936211961664 deprecation.py:323] From /scratch/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1025 17:31:24.105515 139936211961664 deprecation_wrapper.py:119] From /scratch/tensorflow/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/50
  1526/157086 [..............................] - ETA: 16:09 - loss: 0.0088 - categorical_accuracy: 0.0382

KeyboardInterrupt: 

In [56]:
encoder.predict(imageio.imread("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_test16/class0/patch000026.png").reshape(1,16,16,3))

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],
      dtype=float32)

In [57]:
temp_y_prime = encoder.predict(imageio.imread("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_train16/class0/patch000002.png").reshape(1,16,16,3))

In [58]:
temp_y = np.load("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_sifts/tiny_train16/class0/patch000002.npy")

In [59]:
for i in range(temp_y_prime.shape[1]):
    print(temp_y_prime[0, i], temp_y[i])

1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 28.0
1.0 8.0
1.0 0.0
1.0 0.0
1.0 2.0
1.0 45.0
1.0 173.0
1.0 167.0
1.0 48.0
1.0 14.0
1.0 1.0
1.0 1.0
1.0 5.0
1.0 104.0
1.0 173.0
1.0 173.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 46.0
1.0 8.0
1.0 0.0
1.0 0.0
1.0 2.0
1.0 31.0
1.0 173.0
1.0 161.0
1.0 82.0
1.0 13.0
1.0 1.0
1.0 0.0
1.0 4.0
1.0 60.0
1.0 173.0
1.0 173.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0

In [21]:
np.round(temp, decimals=9)

array([[0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 3.427e-06, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+0